In [7]:
import argparse
import pandas as pd
import os

os.chdir('/Users/yanting/OneDrive/Desktop/2023_11_18 Hackathon/2023_11_18-SE-Europe-Data_Challenge')

os.chdir('./src')
import data_processing

os.chdir('/Users/yanting/OneDrive/Desktop/2023_11_18 Hackathon/2023_11_18-SE-Europe-Data_Challenge')

In [2]:
df_dict = data_processing.load_data('/Users/yanting/OneDrive/Desktop/2023_11_18 Hackathon/2023_11_18-SE-Europe-Data_Challenge/data')

In [3]:
df_dict_clean = data_processing.clean_data(df_dict)

In [14]:
regions = {
    'HU': '10YHU-MAVIR----U',
    'IT': '10YIT-GRTN-----B',
    'PO': '10YPL-AREA-----S',
    'SP': '10YES-REE------0',
    'UK': '10Y1001A1001A92E',
    'DE': '10Y1001A1001A83F',
    'DK': '10Y1001A1001A65H',
    'SE': '10YSE-1--------K',
    'NE': '10YNL----------L',
}

reversed_regions = {v: k for k, v in regions.items()}

def preprocess_data(df_dict):
    
    """ 
    Generates new features, transform existing features, resampling & aggregate, etc. 
    """

    df_processed = pd.DataFrame()

    for df_name, df in df_dict.items(): 

        if (df.empty) :
            print('-'*15)
            print(f'{df_name} is empty')
            print('-'*15)
            continue
        
        # all csv files (gen and load) have column AreaID and UnitName
        AreaID = df['AreaID'][0]
        country = reversed_regions[AreaID]
        # country = df_name.str.split('_').str.get(1) # alternative
        UnitName = df['UnitName'][0]
        
        # determine whether its gen or load file
        if df_name.startswith('load') : pwr_type = 'load'
        else : pwr_type = PsrType = df['PsrType'][0]

        # resample (aggregate) to hourly level and sum
        df_resampled = df.resample("1h", label="left").sum()

        # assign new column name including country, power type and unit information
        new_name = f"{country}_{UnitName}_{pwr_type}"
        
        print(f"df name = {df_name}")
        print(f"column name = {new_name}")
        
        df_resampled.rename(columns={'quantity': new_name}, inplace=True)

        # concatenate to output dataframe
        df_processed = pd.concat([df_processed, df_resampled], axis=1)

    return df_processed

In [15]:
df_prepro = preprocess_data(df_dict)

df name = load_SP


UnboundLocalError: local variable 'new_name' referenced before assignment

In [6]:
print(df_prepro.columns)
df_prepro.head()

Index(['Load', 'DE_MAW_B09', 'DE_MAW_B20', 'Load', 'NE_MAW_B20', 'NE_MAW_B18',
       'NE_MAW_B19', 'Load', 'DE_MAW_B18', 'DE_MAW_B19',
       ...
       'NE_MAW_B05', 'NE_MAW_B11', 'Load', 'DE_MAW_B12', 'DE_MAW_B06',
       'NE_MAW_B04', 'DE_MAW_B05', 'DE_MAW_B11', 'DE_MAW_B10', 'DE_MAW_B04'],
      dtype='object', length=114)


,Load,DE_MAW_B09,DE_MAW_B20,Load,NE_MAW_B20,NE_MAW_B18,NE_MAW_B19,Load,DE_MAW_B18,DE_MAW_B19,...,NE_MAW_B05,NE_MAW_B11,Load,DE_MAW_B12,DE_MAW_B06,NE_MAW_B04,DE_MAW_B05,DE_MAW_B11,DE_MAW_B10,DE_MAW_B04
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-31 23:00:00+00:00,20827,26,333,15331.0,2440,1998,1516,3314.0,5688,24850,...,2302,0,4254,119,279,1776,2053,1424,488,2634
2022-01-01 00:00:00+00:00,19530,104,1332,15331.0,9623,6884,5739,3218.0,23112,95800,...,8830,0,16457,378,1116,7470,8184,5690,2274,10513
2022-01-01 01:00:00+00:00,18383,106,1332,15270.0,9357,5518,5494,3126.0,21798,91464,...,8255,0,15426,132,1116,7562,8119,5683,1392,10506
2022-01-01 02:00:00+00:00,17680,108,1332,15150.0,8688,5512,5684,3080.0,21818,86282,...,7327,0,14781,167,1116,6635,8088,5685,710,10585
2022-01-01 03:00:00+00:00,17396,108,1332,15387.0,8375,5668,5492,3044.0,21773,80912,...,6801,0,14630,299,1116,6704,8043,5676,385,10573
